# Paso 0: Creación de bases de datos y esquemas 

## Librerias

In [1]:
# Generales
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')

# Funciones Snowflake
import funciones as snow_func

In [2]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

### Snowflake

In [3]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector # [pip install snowflake-connector-python]
from snowflake.connector.pandas_tools import write_pandas # [pip install "snowflake-connector-python[pandas]"]
from snowflake.snowpark import Session

In [4]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\Conn")
json_file_path = os.path.join(desktop_path, "snowflake_credentials.json")

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
        "account": credentials["ACCOUNT_SNOWFLAKE"],
        "user": credentials["USER_SNOWFLAKE"],
        "password": credentials["PASSWORD_SNOWFLAKE"],
        "role": credentials["ROLE_SNOWFLAKE"],
        "warehouse": credentials["WAREHOUSE"]
    }

# Paso 5: Crear un objeto de conexión utilizando snowflake.connector
session = Session.builder.configs(connection_parameters).create()
print("Sesión actual:", {session})

Sesión actual: {<snowflake.snowpark.session.Session object at 0x000001F7A321D690>}


In [5]:
# Crear objeto de conexión
conn = session.connection

## 1. Crear base de datos

In [6]:
# Query para crear la base de datos
sql_database = """
CREATE OR REPLACE DATABASE DOCUMENTOS_COLOMBIA;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_database)

,status
0,Database DOCUMENTOS_COLOMBIA successfully created.


In [7]:
# Query para identificar ubicación actual
ubicacion = "SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;"
# Ejecutar
snow_func.snowflake_sql(conn, ubicacion)

,WAREHOUSE,DATABASE,SCHEMA
0,WH_PROCOLOMBIA_ANALITICA,DOCUMENTOS_COLOMBIA,PUBLIC


## 2. Crear esquemas

In [8]:
# Crear esquema exportaciones:
sql_schema_exportaciones = """
CREATE OR REPLACE SCHEMA EXPORTACIONES;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_exportaciones)

,status
0,Schema EXPORTACIONES successfully created.


In [9]:
# Crear esquema inversión:
sql_schema_inversion = """
CREATE OR REPLACE SCHEMA INVERSION;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_inversion)

,status
0,Schema INVERSION successfully created.


In [10]:
# Crear esquema turismo:
sql_schema_turismo = """
CREATE OR REPLACE SCHEMA TURISMO;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_turismo)

,status
0,Schema TURISMO successfully created.


In [11]:
# Crear esquema geografia:
sql_schema_geo = """
CREATE OR REPLACE SCHEMA GEOGRAFIA;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_geo)

,status
0,Schema GEOGRAFIA successfully created.


In [12]:
# Crear esquema PARAMETROS:
sql_schema_param = """
CREATE OR REPLACE SCHEMA PARAMETROS;
"""
snow_func.snowflake_sql(conn, sql_schema_param)

,status
0,Schema PARAMETROS successfully created.


In [13]:
# Verificar la estructura de la base de datos
# Crear esquema PARAMETROS:
sql_list_schema = """
SHOW SCHEMAS;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_list_schema)

,created_on,name,is_default,is_current,database_name,owner,comment,options,retention_time,owner_role_type,budget
0,2024-11-21 12:30:51.367000-08:00,EXPORTACIONES,N,N,DOCUMENTOS_COLOMBIA,ANALYTICS,,,1,ROLE,None
1,2024-11-21 12:30:52.065000-08:00,GEOGRAFIA,N,N,DOCUMENTOS_COLOMBIA,ANALYTICS,,,1,ROLE,None
2,2024-11-21 12:30:52.564000-08:00,INFORMATION_SCHEMA,N,N,DOCUMENTOS_COLOMBIA,,Views describing the contents of schemas in this database,,1,,None
3,2024-11-21 12:30:51.578000-08:00,INVERSION,N,N,DOCUMENTOS_COLOMBIA,ANALYTICS,,,1,ROLE,None
4,2024-11-21 12:30:52.313000-08:00,PARAMETROS,N,Y,DOCUMENTOS_COLOMBIA,ANALYTICS,,,1,ROLE,None
5,2024-11-21 12:30:50.878000-08:00,PUBLIC,N,N,DOCUMENTOS_COLOMBIA,ANALYTICS,,,1,ROLE,None
6,2024-11-21 12:30:51.790000-08:00,TURISMO,N,N,DOCUMENTOS_COLOMBIA,ANALYTICS,,,1,ROLE,None


### 2.1 Crear esquema y tabla para realizar seguimiento

In [6]:
# Ejecutar
snow_func.snowflake_sql(conn, "USE DATABASE DOCUMENTOS_COLOMBIA")

,status
0,Statement executed successfully.


In [7]:
# Verificar la estructura de la base de datos
# Crear esquema SEGUIMIENTO:
sql_seguimiento_schema = """
CREATE OR REPLACE SCHEMA SEGUIMIENTO;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_seguimiento_schema)

,status
0,Schema SEGUIMIENTO successfully created.


In [8]:
# Query para identificar ubicación actual
ubicacion = "SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;"
# Ejecutar
snow_func.snowflake_sql(conn, ubicacion)

,WAREHOUSE,DATABASE,SCHEMA
0,WH_PROCOLOMBIA_ANALITICA,DOCUMENTOS_COLOMBIA,SEGUIMIENTO


In [9]:
# Crear tabla de SEGUIMIENTO de eventos principal:
sql_tabla_seguimiento = """
CREATE TABLE SEGUIMIENTO_EVENTOS (
    TIPO_EVENTO STRING,
    DETALLE_EVENTO STRING,
    UNIDAD STRING,
    CORREO STRING,
    TIPO_BOTON STRING,
    FECHA_HORA TIMESTAMP
);
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_tabla_seguimiento)

,status
0,Table SEGUIMIENTO_EVENTOS successfully created.


## 3. Cerrar conexión

In [10]:
conn.close()
session.close()